In [6]:
import pandas as pd
import numpy as np
import os, pdb, sys, pickle
import matplotlib.pyplot as plt
import scipy.stats as stats

with open('results_2.pkl', 'rb') as f: 
    dct = pickle.load(f)
print(dct.keys())
print(dct['chg_var'].keys())
dct['chg_var']['0.001-30'].head()

dict_keys(['chg_var', 'cov_shift', 'mix_shift'])
dict_keys(['0.0001-30', '0.001-30', '0.01-30', '0.1-30', '0.6-30', '1-30', '5-30', '10-30', '20-30', '30-30'])


,x1,x2,x3,y,time,pe
0,0.496714,1.399355,-0.675178,1.220891,0,0.0
1,-0.138264,0.924634,-0.144519,0.641851,1,0.0
2,0.647689,0.059630,-0.792420,-0.085101,2,0.0
3,1.523030,-0.646937,-0.307962,0.568132,3,0.0
4,-0.234153,0.698223,-1.893615,-1.429545,4,0.0


In [16]:
for typeOfShift in dct.keys(): 
    res = dct[typeOfShift]
    df = None
    for k in res.keys(): 
        if df is None: 
            df = res[k][['time', 'y', 'x1']]
            df.columns = ['time', 'y', 'shift_' + str(k).split('-')[0]]
        else: 
            tmp = res[k][['time', 'x1']]
            tmp.columns = ['time', 'shift_' + str(k).split('-')[0]]
            df = pd.merge(df, tmp, how = 'left', on = 'time')
    
    for col in [i for i in df.columns if 'shift' in i]: 
        # Apply Traditional Drift Metrics for Thresholding
        # KS Test, Chi-Squared Test, PSI
        ks, psi = [], []
        b1, b2 = 0, 400
        a1, a2 = 401, 501
        while a2 < len(df): 
            ds1 = df[[col, 'y']].iloc[b1:b2+1]
            ds2 = df[[col, 'y']].iloc[a1:a2+1]

            # Compute KS
            pdb.set_trace()
            ks_pval = stats.ks_2samp(ds1, ds2).pvalue
            ks.append(ks_pval)

            # Compute PSI
            hist1, _ = np.histogram(ds1, bins=10)
            hist2, _ = np.histogram(ds2, bins=10)
            hist1 = hist1/sum(hist1)
            hist2 = hist2/sum(hist2)
            psi_val = np.sum((hist1-hist2) * np.log(hist1/hist2))
            psi.append(psi_val)

            # Iterate
            b1 += 10
            b2 += 10
            a1 += 10
            a2 += 10

        result = pd.DataFrame({'ks': ks, 'psi': psi})
        pdb.set_trace()

> c:\users\cshul\appdata\local\temp\ipykernel_11512\2987445153.py(25)<module>()

   shift_0.0001         y
0      0.496714  1.220891
1     -0.138264  0.641851
2      0.647689 -0.085101
3      1.523030  0.568132
4     -0.234153 -1.429545
*** ValueError: object too deep for desired array
